In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
train_df.head()

In [ ]:
target = train_df['label']
y_train = np.zeros([42000,10])
count=0
for i in target:
    y_train[count,i] = 1
    count = count+1
train_df = train_df.drop(['label'],axis=1)
X_train = train_df.as_matrix()
X_train = X_train.reshape((X_train.shape[0],28,28,1))

In [ ]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
plt.subplot(131)
plt.title('label = 1')
plt.imshow(X_train[0,:,:,0],cmap='gray')

plt.subplot(132)
plt.title('label = 0')
plt.imshow(X_train[5,:,:,0],cmap='gray')

plt.subplot(133)
plt.title('label = 8')
plt.imshow(X_train[10,:,:,0],cmap='gray')

plt.show()

In [ ]:
def digit_model(input_shape):
    X_in = Input(input_shape)
    
    X = Conv2D(16,kernel_size=(3,3),padding='same',input_shape=(28,28,1))(X_in)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2))(X)
    
    X = Conv2D(32,kernel_size=(3,3),padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2))(X)
    
    X = Conv2D(64,kernel_size=(3,3),padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2))(X)
    
    X = Flatten()(X)
    
    X = Dense(128)(X)
    X = Activation('relu')(X)
    
    X = Dense(64)(X)
    X = Activation('relu')(X)
    
    X = Dense(10)(X)
    X = Activation('softmax')(X)
    
    model = Model(inputs = X_in,outputs = X , name='digit_model')
    return model

In [ ]:
my_model = digit_model((28,28,1))
print(my_model)

In [ ]:
X_train = X_train/255
my_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
my_model.fit(x=X_train,y=y_train,epochs=20,batch_size=256)

In [ ]:
my_model.evaluate(x=X_train,y=y_train)

In [ ]:
my_model.save('my_model_digit.h5')

In [ ]:
X_test = test_df.as_matrix()/255
X_test = X_test.reshape((X_test.shape[0],28,28,1))

In [ ]:
prob = my_model.predict(x = X_test)
predlabel = np.argmax(prob,axis=1)
sub_df = pd.DataFrame(predlabel,columns = ['Label'])
sub_df.reset_index(inplace=True)
sub_df.rename(columns={'index': 'ImageId'}, inplace=True)
sub_df['ImageId']=sub_df['ImageId']+1
sub_df.to_csv('output.csv', index=False)